In [ ]:
import cv2
import json
from ultralytics import YOLO  # Adjust the import according to how you import your model
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Load your YOLOv8 model
model = YOLO('/Users/apaula/Downloads/model_- 2 january 2024 18_31.pt')

def detect_and_draw_boxes_on_tile(image_path, model, tile_size=640, start_x=50, start_y=50):
    # Load the whole image
    image = cv2.imread(image_path)

    # Extract a 216x216 tile from the image
    tile = image[start_y:start_y + tile_size, start_x:start_x + tile_size]

    # Save the tile temporarily (required for your model's input format)
    temp_tile_path = 'temp_tile.jpg'
    cv2.imwrite(temp_tile_path, tile)

    # Run detection on the tile
    results = model(temp_tile_path)
    json_results = json.loads(results[0].tojson())

    # Prepare to draw bounding boxes on the tile using Matplotlib
    fig, ax = plt.subplots(1, figsize=(8, 8))
    ax.imshow(cv2.cvtColor(tile, cv2.COLOR_BGR2RGB))  # Convert tile to RGB for Matplotlib

    for detection in json_results:
        box = detection['box']
        x1, y1, x2, y2 = int(box['x1']), int(box['y1']), int(box['x2']), int(box['y2'])
        rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)

    plt.show()

    # Cleanup: Delete the temporary tile file
    import os
    os.remove(temp_tile_path)

# Path to your whole picture
image_path = '/Users/apaula/Downloads/testframe.jpg'
detect_and_draw_boxes_on_tile(image_path, model)


In [ ]:
import cv2
import json
from ultralytics import YOLO  # Adjust the import according to how you import your model
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Load your YOLOv8 model
model = YOLO('/Users/apaula/Downloads/model_- 2 january 2024 18_31.pt')

def tile_image(image, tile_size=640, overlap=0.2):
    """
    Tile the image into smaller overlapping tiles.
    Returns a list of tiles and their positions in the original image.
    """
    tiles = []
    step = int(tile_size * (1 - overlap))

    # Determine the starting point for the last tile in both dimensions
    max_y = image.shape[0] - tile_size
    max_x = image.shape[1] - tile_size

    for y in range(0, image.shape[0], step):
        for x in range(0, image.shape[1], step):
            # Adjust the tile size if we are at the edge of the image
            adjusted_tile_size_y = min(tile_size, image.shape[0] - y)
            adjusted_tile_size_x = min(tile_size, image.shape[1] - x)

            tile = image[y:y + adjusted_tile_size_y, x:x + adjusted_tile_size_x]
            tiles.append((tile, (x, y)))

    return tiles


def detect_objects_on_tiles(tiles, model):
    all_boxes = []
    for tile, position in tiles:
        # Save the tile temporarily (required for your model's input format)
        temp_tile_path = 'temp_tile.jpg'
        cv2.imwrite(temp_tile_path, tile)

        # Run detection on the tile
        results = model(temp_tile_path)
        json_results = json.loads(results[0].tojson())

        # Adjust box coordinates based on the tile position
        for detection in json_results:
            box = detection['box']
            x1, y1, x2, y2 = int(box['x1']), int(box['y1']), int(box['x2']), int(box['y2'])
            all_boxes.append((x1 + position[0], y1 + position[1], x2 + position[0], y2 + position[1]))

        # Cleanup: Delete the temporary tile file
        import os
        os.remove(temp_tile_path)
    
    return all_boxes

def draw_boxes_on_frame(frame, boxes):
    fig, ax = plt.subplots(1, figsize=(40, 30))
    ax.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))  # Convert frame to RGB for Matplotlib

    for box in boxes:
        x1, y1, x2, y2 = box
        rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)

    plt.show()

# Path to your whole picture
image_path = '/Users/apaula/Downloads/testframe.jpg'
frame = cv2.imread(image_path)

# Tile the image and detect objects
tiles = tile_image(frame)
boxes = detect_objects_on_tiles(tiles, model)

# Draw boxes on the whole frame
draw_boxes_on_frame(frame, boxes)


In [ ]:
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from ultralytics import YOLO  # Adjust the import according to your model setup

# Load your YOLOv8 model
model = YOLO('/Users/apaula/Downloads/model_- 2 january 2024 18_31.pt')

def tile_image(image, tile_size=640, overlap=0.2):
    """
    Tile the image into smaller overlapping tiles.
    Returns a list of tiles and their positions in the original image.
    """
    tiles = []
    step = int(tile_size * (1 - overlap))

    # Determine the starting point for the last tile in both dimensions
    max_y = image.shape[0] - tile_size
    max_x = image.shape[1] - tile_size

    for y in range(0, image.shape[0], step):
        for x in range(0, image.shape[1], step):
            # Adjust the tile size if we are at the edge of the image
            adjusted_tile_size_y = min(tile_size, image.shape[0] - y)
            adjusted_tile_size_x = min(tile_size, image.shape[1] - x)

            tile = image[y:y + adjusted_tile_size_y, x:x + adjusted_tile_size_x]
            tiles.append((tile, (x, y)))

    return tiles

def detect_objects_on_tiles(tiles, model):
    all_boxes = []
    all_classes = []
    for tile, position in tiles:
        # Save the tile temporarily (required for your model's input format)
        temp_tile_path = 'temp_tile.jpg'
        cv2.imwrite(temp_tile_path, tile)

        # Run detection on the tile
        results = model(temp_tile_path)
        json_results = json.loads(results[0].tojson())

        # Adjust box coordinates and extract class information based on the tile position
        for detection in json_results:
            box = detection['box']
            x1, y1, x2, y2 = int(box['x1']), int(box['y1']), int(box['x2']), int(box['y2'])
            all_boxes.append((x1 + position[0], y1 + position[1], x2 + position[0], y2 + position[1]))
            all_classes.append(detection['class'])  # Assuming 'class' is the key for class information

        # Cleanup: Delete the temporary tile file
        import os
        os.remove(temp_tile_path)
    
    return all_boxes, all_classes

def draw_boxes_on_frame_with_color_wheel(frame, boxes, classes, class_labels, colors):
    # Create a subplot layout
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(120, 60), gridspec_kw={'width_ratios': [3, 1]})

    # Plot the frame with bounding boxes
    ax1.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))  # Convert frame to RGB for Matplotlib
    for box, cls in zip(boxes, classes):
        x1, y1, x2, y2 = box
        color = colors[cls % len(colors)]
        rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=2, edgecolor=color, facecolor='none')
        ax1.add_patch(rect)
    ax1.axis('off')

    # Create the color wheel in the second subplot
    num_classes = len(class_labels)
    label_to_angle = {'east': 0, 'northeast': np.pi/4, 'north': np.pi/2, 'northwest': 3*np.pi/4,
                      'west': np.pi, 'southwest': 5*np.pi/4, 'south': 3*np.pi/2, 'southeast': 7*np.pi/4}
    angles = [label_to_angle[label] for label in class_labels]
    ax2 = plt.subplot(122, polar=True)
    ax2.set_theta_zero_location('E')
    ax2.set_theta_direction(1)
    ax2.set_xticks(angles)
    ax2.set_xticklabels(class_labels)
    ax2.set_yticklabels([])
    for ang, color in zip(angles, colors):
        ax2.bar(ang, 1, width=2*np.pi/num_classes, color=color, bottom=0.4)

    plt.tight_layout()
    plt.show()

# Path to your whole picture
image_path = '/Users/apaula/Downloads/testframe.jpg'
frame = cv2.imread(image_path)

# Tile the image and detect objects
tiles = tile_image(frame)
boxes, classes = detect_objects_on_tiles(tiles, model)

# Class labels and their corresponding colors
class_labels = ['east', 'north', 'northeast', 'northwest', 'south', 'southeast', 'southwest', 'west']
colors = [
    (1.0, 0.0, 0.0),       # East - Red
    (0.31, 1.0, 0.0),      # North - Bright Green
    (1.0, 0.84, 0.0),      # Northeast - Gold
    (0.0, 1.0, 0.529),     # Northwest - Turquoise
    (0.996, 0.0, 0.937),   # South - Pink
    (1.0, 0.0, 0.09),      # Southeast - Scarlet
    (0.216, 0.0, 1.0),     # Southwest - Electric Blue
    (0.0, 0.624, 1.0)      # West - Sky Blue
]

# Draw colored boxes on the whole frame and the color wheel
draw_boxes_on_frame_with_color_wheel(frame, boxes, classes, class_labels, colors)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def draw_color_wheel(class_labels, colors):
    num_classes = len(class_labels)

    # Map each class to an angle on the wheel: North (0°), East (90°), South (180°), West (270°)
    label_to_angle = {
        'North': 0,
        'Northeast': np.pi/4,
        'East': np.pi/2,
        'Southeast': 3*np.pi/4,
        'South': np.pi,
        'Southwest': 5*np.pi/4,
        'West': 3*np.pi/2,
        'Northwest': 7*np.pi/4
    }

    angles = [label_to_angle[label] for label in class_labels]

    fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
    ax.set_theta_zero_location('N')
    ax.set_theta_direction(-1)  # Clockwise
    ax.set_xticks(angles)
    ax.set_xticklabels(class_labels)
    ax.set_yticklabels([])

    for ang, color in zip(angles, colors):
        ax.bar(ang, 1, width=2*np.pi/num_classes, color=color, bottom=0.4)

    plt.show()

# Example usage
class_labels = ['North', 'Northeast', 'East', 'Southeast', 'South', 'Southwest', 'West', 'Northwest']
colors = [(0.31, 1.0, 0.0), (1.0, 0.84, 0.0), (1.0, 0.0, 0.0), (1.0, 0.0, 0.09), (0.996, 0.0, 0.937), (0.216, 0.0, 1.0), (0.0, 0.624, 1.0), (0.0, 1.0, 0.529)]

draw_color_wheel(class_labels, colors)
